# **Suzuki-Trotter Error Check**

## General Setup

Notebook is structured such that one can "run all". Or change parameters and then "run all" assuming all packages are installed

### Imports

In [1]:
from qiskit import *
import numpy as np
from qutip import *
from matplotlib import *
from pylatexenc import *
from matplotlib import pyplot as plt
from qiskit.tools.jupyter import *
import qiskit.quantum_info as qi
import itertools
from tqdm import tqdm

### Parameter Definitions

In [27]:
n = 5
tau = 0.5
gamma = 0.05
omega = 1
T = 2*np.pi/omega
epsilon = gamma/T
lamb = gamma/T
J = tau/T
h = 2*(np.pi/2 + epsilon)/T

tmax = 60
dt = 0.1
steps = int(tmax//dt)
print(steps)

599


## Qutip

In [28]:
dim = [2] * n
lanes = list(range(n))

def QubitOperation(U,i,n):
    Q = tensor(qeye(2**(i-1)),U,qeye(2**(n-i)))
    return Q

def PauliX(i,n):
    X = Qobj(QubitOperation(sigmax(),i,n),dims=[[2**n],[2**n]])
    return X

def PauliY(i,n):
    Y = Qobj(QubitOperation(sigmay(),i,n),dims=[[2**n],[2**n]])
    return Y

def PauliZ(i,n):
    Z = Qobj(QubitOperation(sigmaz(),i,n),dims=[[2**n],[2**n]])
    return Z

def PauliP(i,n):
    P = (PauliX(i,n) + 1j*PauliY(i,n))/2
    return P

def PauliM(i,n):
    M = (PauliX(i,n) - 1j*PauliY(i,n))/2
    return M

def H2_coeff(t,args):
    return -h*np.cos(omega*t/2)**2

def QO(l):
    return Qobj(l,dims=[[2],[1]])

In [29]:
H0 = PauliY(1,n) + PauliZ(1,n)
for i in range(2,n+1):
    H0 = H0 + PauliY(i,n) + PauliZ(i,n)
H1 = (PauliZ(1,n) * PauliZ(2,n))
for i in range(2,n):
    H1 = H1 + (PauliZ(i,n) * PauliZ(i+1,n))
H2 = PauliX(1,n)
for i in range(2,n+1):
    H2 = H2 + PauliX(i,n)
EX = (1/n)*PauliX(1,n)
for i in range(2,n+1):
    EX = EX + (1/n)*PauliX(i,n)
EY = (1/n)*PauliY(1,n)
for i in range(2,n+1):
    EY = EY + (1/n)*PauliY(i,n)
EZ = (1/n)*PauliZ(1,n)
for i in range(2,n+1):
    EZ = EZ + (1/n)*PauliZ(i,n)

In [30]:
psi_states_list = list(itertools.product([(1,1j), (1,-1j)], repeat=n))
psi_states = 2**n * [1]
for m in range(2**n):
    psi_states[m] = tensor(QO(psi_states_list[m][0]),QO(psi_states_list[m][1]))
    for i in range(2,n):
        psi_states[m] = tensor(psi_states[m],QO(psi_states_list[m][i]))
    psi_states[m] = Qobj(psi_states[m].trans().unit(),dims=[[2**n],[1]])

In [31]:
t = np.linspace(0, tmax, steps)

H = [lamb*H0,-J*H1,[H2, H2_coeff]]

qt_state = []
for i in tqdm(range(2**n)):
    qt_state.append(mesolve(H, psi_states[i], t))

100%|██████████| 32/32 [00:03<00:00,  8.69it/s]


## Qiskit

### Define Hamiltonian

In [32]:
def coeff(t):
    return -h*np.cos(omega*t/2)**2

def hamiltonian(t):
        #H0 terms with sigma^y and sigma^z
        for i in range(n):
            qc.ry(2*lamb*dt,i)
            qc.rz(2*lamb*dt,i)
        qc.barrier()
        #H1 terms with sigma^z
        for i in range(n-1):
            qc.cx(i,i+1)
            qc.rz(-2*J*dt,i+1)
            qc.cx(i,i+1)
        qc.barrier()
        #H2 time dependent terms with sigma^x
        for i in range(n):
            qc.rx(2*coeff(t)*dt,i)

### Run Quantum Circuit

In [33]:
results = []
for i in tqdm(range(2**n)):
    qc = QuantumCircuit(n)
    psi = qi.Statevector(np.array(psi_states[i]))
    qc.initialize(psi)
    qc.save_state()
    for i in range(steps):
        hamiltonian(i*dt)

    #qc.draw(output='mpl',interactive=True,filename='circuit_drawing.png',fold=1000)
    sim = Aer.get_backend('aer_simulator')
    qobj = assemble(qc)
    qc.save_statevector()
    results.append(np.array(sim.run(qc).result().get_statevector()))

100%|██████████| 32/32 [00:16<00:00,  1.91it/s]


## Calculate Fidelity

In [35]:
def inner_product(i):
    qc_state = Qobj(results[i],dims=[[2**n],[1]])
    return qc_state.overlap(qt_state[i].states[-1]) 

fidelity = 0
for i in range(2**n):
    fidelity += inner_product(i)
fidelity = np.real(fidelity/2**n)
print(fidelity)

0.9982176530617083
